In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import time

In [3]:
quranLink = "https://mp3quran.net/eng/ryan" # note: the web page must have 114 ayas
wikipediaLink = "https://en.wikipedia.org/wiki/List_of_chapters_in_the_Quran"

In [4]:
pageHtml = requests.get(quranLink).text
pageHtml = BeautifulSoup(pageHtml, "html.parser")

In [5]:
print(pageHtml.prettify) # for debugging by visualizing

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="eng">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<!-- CSRF Token -->
<meta content="i2OHCd3xHzseWz2WRr81brbMwghRLUkEOmJAuas7" name="csrf-token"/>
<title>reciter title hafs</title>
<meta content="reciter title hafs" property="og:title">
<meta content="reciter title hafs" name="twitter:title">
<meta content="reciter title hafs" property="og:title"/>
<meta content="reciter description hafs" name="description"/>
<meta content="reciter description hafs" property="og:description"/>
<meta content="reciter description hafs" name="twitter:description"/>
<meta content="read.keywords-46" name="keywords"/>
<meta content="http://mp3quran.net/eng/ryan" property="og:url"/>
<meta content="" property="og:site_name"/>
<meta content="" name="twitter:image"/>
<meta content="summary" name="twitter:card">
<meta content="" nam

In [6]:
surahsDetails = pageHtml.select("card-sora") # for debugging by visualizing
surahsDetails[0:2]

[<card-sora :share=' {"title":"share title read","url":"http:\/\/mp3quran.net\/eng\/ryan","description":"share description read"} ' :sora='{"id":"001-46","sora_id":1,"sora_num":"001","sora_name":"Al-Fatihah ","read_slug":"ryan","sora_report":7,"sora_duration":"0:45","sora_link":"http:\/\/mp3quran.net\/eng\/ryan\/1","sora_audio":"https:\/\/server8.mp3quran.net\/ryan\/001.mp3","share_url":"http:\/\/mp3quran.net\/eng\/ryan\/1","share_title":"share title sora","share_description":"share description sora","show":true}' read_id="46" reciter="Adel Ryyan" rewaya="Rewayat Hafs A'n Assem"></card-sora>,
 <card-sora :share=' {"title":"share title read","url":"http:\/\/mp3quran.net\/eng\/ryan","description":"share description read"} ' :sora='{"id":"002-46","sora_id":2,"sora_num":"002","sora_name":"Al-Baqarah ","read_slug":"ryan","sora_report":3,"sora_duration":"1:53:16","sora_link":"http:\/\/mp3quran.net\/eng\/ryan\/2","sora_audio":"https:\/\/server8.mp3quran.net\/ryan\/002.mp3","share_url":"http:\

In [7]:
surahsDetails[0].attrs # for debugging by visualizing

{':sora': '{"id":"001-46","sora_id":1,"sora_num":"001","sora_name":"Al-Fatihah ","read_slug":"ryan","sora_report":7,"sora_duration":"0:45","sora_link":"http:\\/\\/mp3quran.net\\/eng\\/ryan\\/1","sora_audio":"https:\\/\\/server8.mp3quran.net\\/ryan\\/001.mp3","share_url":"http:\\/\\/mp3quran.net\\/eng\\/ryan\\/1","share_title":"share title sora","share_description":"share description sora","show":true}',
 'rewaya': "Rewayat Hafs A'n Assem",
 'read_id': '46',
 'reciter': 'Adel Ryyan',
 ':share': ' {"title":"share title read","url":"http:\\/\\/mp3quran.net\\/eng\\/ryan","description":"share description read"} '}

In [8]:
surahsDetails = pageHtml.select("card-sora")
json.loads(surahsDetails[0].attrs[':sora']) # json.loads() converts string to dictionary

{'id': '001-46',
 'sora_id': 1,
 'sora_num': '001',
 'sora_name': 'Al-Fatihah ',
 'read_slug': 'ryan',
 'sora_report': 7,
 'sora_duration': '0:45',
 'sora_link': 'http://mp3quran.net/eng/ryan/1',
 'sora_audio': 'https://server8.mp3quran.net/ryan/001.mp3',
 'share_url': 'http://mp3quran.net/eng/ryan/1',
 'share_title': 'share title sora',
 'share_description': 'share description sora',
 'show': True}

In [9]:
wikiSurahs = pd.read_html(wikipediaLink)[0] # for debugging by visualizing


In [10]:
wikiSurahs[wikiSurahs['#'] == 1] # for debugging by visualizing

,#,Anglicized title(s),Arabic title(s),English title(s),Number of verses (Number of Rukūʿs),Place of Revelation,Egyptian Standard Chronological Order[5][6][7],Nöldeke's Chronological Order[5],Muqatta'at (isolated letters)[3],Title refers to,Main theme(s),Juz'
0,1,Al-Fatihah,ٱلْفَاتِحَةal-Fātiḥahal-Ḥamd,"The Opening, the Opening of the Divine Writ, T...",7 (1),Makkah,5,48,NaN,Whole Surah [8],The fundamental principles of the Qur'an in a ...,1


In [11]:
surahDic = json.loads(surahsDetails[0].attrs[':sora'])
mp3FileName = str(surahDic['sora_num']) + " " + wikiSurahs[wikiSurahs['#'] == surahDic['sora_id']]['Anglicized title(s)']
mp3FileName

0    001 Al-Fatihah
Name: Anglicized title(s), dtype: object

In [12]:
type(mp3FileName) # for debugging by visualizing

pandas.core.series.Series

In [13]:
mp3FileName.values[0] + ".mp3" # for debugging by visualizing

'001 Al-Fatihah.mp3'

In [13]:
tryLaterAudios = []
for i in range(0, len(surahsDetails)):
    surahDic = json.loads(surahsDetails[i].attrs[':sora'])
    mp3FileName = str(surahDic['sora_num']) + " " + wikiSurahs[wikiSurahs['#'] == surahDic['sora_id']]['Anglicized title(s)']
    mp3FileName = mp3FileName.values[0] + ".mp3"
    mp3Link = surahDic['sora_audio']
    try:
        mp3 = requests.get(mp3Link, verify = True) # "verify" argument could be set to False to help fix ConnectionError "attempt failed because the connected party did not properly respond after a period of time"
    except:
        tryLaterAudios.append(i)
        continue
    with open(mp3FileName, 'wb') as f:
        f.write(mp3.content) # ".content" accesses the audio of the get request in case you use a link that downloads an mp3 file
    print('.', end = '')
    time.sleep(3) # also to avoid ConnectionError

c:\Users\Gamma\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'server8.mp3quran.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


.

c:\Users\Gamma\AppData\Local\Programs\Python\Python38\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'server8.mp3quran.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


.

In [ ]:
print("\nFinished Downloading!")
print("failed audios: " + tryLaterAudios)

In [ ]:
for idx in tryLaterAudios:
    surahDic = json.loads(surahsDetails[i].attrs[':sora'])
    mp3FileName = str(surahDic['sora_num']) + " " + wikiSurahs[wikiSurahs['#'] == surahDic['sora_id']]['Anglicized title(s)']
    mp3FileName = mp3FileName.values[0] + ".mp3"
    mp3Link = surahDic['sora_audio']
    try:
        mp3 = requests.get(mp3Link, verify = False) # "verify" argument to fix ConnectionError "attempt failed because the connected party did not properly respond after a period of time"
    except:
        continue
    with open(mp3FileName, 'wb') as f:
        f.write(mp3.content) # ".content" accesses the audio of the get request in case you use a link that downloads an mp3 file
    print('.', end = '')
    time.sleep(3) # also to avoid ConnectionError